# Neural Collaborative Filtering  
- paper reivew

## Introduction  
- explicit feedback에 비해 implicit feedback은 추적하기가 쉽고 데이터를 수집하기 쉽다는 장점이 있다  
- 그러나, 유저의 만족도에 대한 정보를 알 수가 없고 negative feedback이 부족해 활용하기가 어렵다는 단점이 존재한다   
- 그래서 이 논문에서는 implicit feedback에 대한 데이터를 neural network를 활용해 학습하고자 한다 
- main contributions:  
    - users와 items의 latent features를 모델링하는 neural network architecture를 제안하고 neural network 기반 collaborative filtering에 대한 general framework NCF를 고안이 되었다
    - MF는 NCF의 specialization으로 설명될 수 있고 non-linearity의 high level을 지니고 있는 multi-layer perceptron을 활용할 수 있음을 보일 것이다  
    - 2개의 실제 데이터셋을 활용해 본 논문에서 제안한 NCF의 효과를 실험으로 보여준다  

## Preliminaries  
### Learning from Implicit Data  
- $M$과 $N$이 각각 user와 item의 갯수라고 가정하자  
- 그러면 user-item 행렬은 1 또는 0으로 이루어진 $M\times N$로 표현할 수 있다  
- element가 1인 경우는 해당 user와 item 사이의 interaction이 있음을 의미하고 이는 user가 item을 열람했거나, 구매했거나 등의 implicit 정보를 의미한다  
- 그러나 '선호한다'는 의미는 아니다  
- 반대로 0인 경우에도 interaction이 없는 것이며 '선호하지 않는다'는 아니다  
- 본 논문에서 implcit feedback 데이터에 대한 recommendation problem을 unobserved entries의 score를 추정하는 문제로 만들어낸다  
- 수식으로 표현하면 다음과 같이 쓸 수 있다  
$$\hat{y}_{u,i}=f(u,i\vert \theta)$$  
- 이때 $\hat{y}_{u,i}$는 predicted score, $\theta$는 model parameters 그리고 f는 interaction function 즉, neural network  
- Recommender system에는 크게 point-wise loss와 pair-wise loss가 쓰이는데, 본 논문의 NCF framework에서는 두 loss function을 모두 사용할 수 있다고 한다  

### Matrix Factorization  
- MF는 간단히 user-item matrix를 더 저차원의 2개의 matrix로 분해해 표현하는 것이다  
- 즉, MF는 interaction $y_{u,i}$를 user latent vector $\textbf{p}_u$와 item latent vector $\textbf{q}_i$의 내적으로 표현할 수 있으며 수식으로 나타내면 다음과 같다  

$$\hat{y}_{u,i}=f(u,i\vert \textbf{p}_u, \textbf{q}_i)=\textbf{p}_u^T \textbf{q}_i=\sum_{k=1}^Kp_{u,k}q_{i,k}$$  
- 이때 $K$는 latent space의 dimension을 의미한다  

<img src = "https://github.com/Sangh0/Recommender-System/blob/main/NCF/figures/figure1.png?raw=true" width=400>  

- Matrix Factorization의 한계점을 살펴보자  
- 예를 들어 위의 그림의 a와 같이 user-item matrix가 있다고 할 때 $s_{ij}$는 user $i$와 user $j$의 유사도를 나타낸다고 하자   
- $s_{23}(0.66) > s_{12}(0.5) > s_{13}(0.4)$ 이와 같은 관계를 얻을 수 있다  
- 즉, user 2와 3이 가장 비슷하고 user 1과 3이 가장 덜 비슷하다는 뜻이다  
- 이를 기하학적으로 표현하면 위 그림의 b로 볼 수 있다  
- 만약 여기서 새로운 user 4가 나타났을 때, 그림 a에 따르면 다음과 같은 관계를 볼 수 있다  
- $s_{41}(0.6) > s_{43}(0.4) > s_{42}(0.2)$  
- 그러나 이 관계는 위 그림의 b에서 표현을 할 수가 없다  
- 즉, user 4와 1을 가장 가깝게 하면서 user 4와 2를 가장 멀게 하는 vector $\textbf{p}_4$를 찾을 수 없다  
- 다시 말해, inner product는 복잡한 관계를 표현하기 힘들다는 한계가 있다  
- 그래서 본 논문에서는 neural network를 이용해 이를 해결하고자 한다  

## Neural Collaborative Filtering  

<img src = 'https://github.com/Sangh0/Recommender-System/blob/main/NCF/figures/figure2.png?raw=true'>  

- 위 그림은 본 논문에서 제안한 NCF의 general framework이다  
- input layer  
    - 2개의 feature인 $\textbf{v}_u^U$와 $\textbf{v}_i^I$로 이루어져 있으며 각각 user $u$와 item $i$를 의미함  
    - binarized sparse vector인 one hot encoding으로 표현  
    
- embedding layer  
    - fc layer로 구성되어 있으며 sparse representation을 dense vector로 projection하는 역할을 수행  
    - user or item embedding은 latent vector로 볼 수 있다  
    - embedding vector는 NCF라고도 하는 MLP에 입력으로 넣는다   
 
- Neural CF layers  
    - user latent vector와 item latent vector를 concatenation을 수행해 입력으로 넣으며 마지막 히든 레이어는 모델의 사이즈를 결정하며 sore $\hat{y}_{u,i}$를 예측한다   
    
### Learning NCF  
- 보통 regression 문제에서 squared loss를 많이 사용한다  
- squared loss는 observation이 gaussian distribution에서 생성되었다고 가정함으로써 설명할 수 있다  
- 그러나 우리는 implicit data를 사용하며, target은 0 or 1로 표현하므로 binary property에 주의를 해야 한다  
- 따라서 우리는 likelihood function은 다음과 같이 정의한다  
$$p(y, y^{-}\vert \textbf{P}, \textbf{Q}, \theta)=\prod_{(u,i)\in y}\hat{y}_{u,i}\prod_{(u,j)\in y^{-}}(1-\hat{y}_{u,j})$$  
- loss function은 다음과 같다  
$$L=-\sum_{u,i}\in y log \hat{y}_{u,i} - \sum_{u,j}\in y^{-} log(1-\hat{y}_{u,j})$$
$$=-\sum_{(u,i)\in y\cup y^{-} y_{u,i}log \hat{y}_{u,i}} + (1- y_{u,i}) log (1- \hat{y}_{u,i})$$  

- 직관적으로, 위 로스는 BCE를 사용하는 것이며 binary classification 문제로 귀결된다  
- NCF에 대한 objective function이며 이는 SGD로 최적화된다  

### Generalized Matrix Factorization (GMF)  
- 본 논문은 MF가 NCF의 특별한 케이스가 될 수 있음을 보여주며 이를 GMF라고 한다  
$$\hat{y}_{u,i}=a_{out}\left(h^T\phi_1 (\textbf{p}_u, \textbf{q}_i)\right)$$  
- 이때 $a_{out}$은 identity function, $h^T=\left[1,..., 1\right]_{1\times k}$  
- 이를 MF라고 하며 GMF는 $a_{out}$과 $h^T$를 아래와 같이 설정해 일반화된 버전이다  
$$a_{out}=\frac{1}{1+e^{-x}},\quad h^T=\left[h_1, ..., h_k\right]$$  

### Multi-Layer Perceptron  
- GMF는 linear하고 fixed한 특징을 가지는 반면 MLP는 non-linear하고 flexible하기 때문에 복잡한 관계를 표현할 수 있음  
- MLP의 수식화는 간단하니 pass  
- activation에 대한 얘기를 하자면  
    - sigmoid는 출력값을 0과 1사이의 값으로 제한하기 때문에 gradient vanishing 문제를 일으킨다  
    - tanh는 sigmoid 출력 범위의 문제를 어느 정도 해결하지만 그래도 비슷하다  
    - ReLU는 sparse data에 잘 피팅되며 오버피팅 날 가능성도 적다  
- architecture에 대한 얘기를 하자면, tower pattern으로 구성해야 한다.
- 즉, 각 레이어는 이전 레이어의 노드 갯수보다 적어야 한다  

### Fusion of GMF and MLP  
<img src = "https://github.com/Sangh0/Recommender-System/blob/main/NCF/figures/figure3.png?raw=true">  

- 본 논문에서 위 그림과 같이 GMF와 MLP를 통합한 모델을 제안한다  
- 이는 Neural Matrix Factorization 즉, NeuMF라 명명한다  
- 가장 큰 특징으로 각 모델별로 서로 다른 embedding layer를 사용하는 것이다  

#### Pre-training  
- NeuMF를 학습할 때는 GMF와 MLP는 사전학습된 모델로 초기화한다  
- 그 전에 GMF와 MLP를 학습할 때는 random initialization으로 수행   
- 또한 GMF와 MLP를 학습할 때는 Adam optimizer로 수행하며 NeuMF를 학습할 때는 SGD로 학습한다  